Задание 1

Для датафрейма log из материалов занятия создайте столбец source_type по следующим правилам:

если источник traffic_source равен yandex или google, то в source_type ставится organic
для источников paid и email из России - ставим ad
для источников paid и email не из России - ставим other
все остальные варианты берем из traffic_source без изменений

In [2]:
import pandas as pd
log = pd.read_csv('visit_log.csv', sep=';')
log['source_type'] = log.loc[(log['traffic_source'] == 'paid') | (log['traffic_source'] == 'email') ].apply(lambda row: 'ad' if 'Russia' in row.region else 'other', axis=1)
log.loc[log.traffic_source == 'yandex', 'source_type'] = 'organic'
log.loc[log.traffic_source == 'google', 'source_type'] = 'organic'
log.loc[log['source_type'].isnull(),'source_type'] = log['traffic_source']

    
print(log)


        timestamp    visit_id                                 url   region  \
0      1549980692  e3b0c44298  https://host.ru/3c19b4ef7371864fa3   Russia   
1      1549980704  6e340b9cff  https://host.ru/c8d9213a31839f9a3a   Russia   
2      1549980715  96a296d224  https://host.ru/b8b58337d272ee7b15   Russia   
3      1549980725  709e80c884  https://host.ru/b8b58337d272ee7b15   Russia   
4      1549980736  df3f619804  https://host.ru/b8b58337d272ee7b15   Russia   
...           ...         ...                                 ...      ...   
18933  1550094288  57e5ba8560  https://host.ru/c2382eb3d6afc8d0f3  Belarus   
18934  1550094296  6f9389ec1b  https://host.ru/f1eb4601740d627ab0   Russia   
18935  1550094308  e8cf2eb8e6  https://host.ru/a5dda93e70318570c0  Belarus   
18936  1550094314  79530b9a67  https://host.ru/6fda01ec57f23abc9e   Russia   
18937  1550094323  b3b634f824  https://host.ru/39fa884393666d45fc   Russia   

          user_id traffic_source source_type  
0      b1613cc09

Задание 2

В файле URLs.txt содержатся url страниц новостного сайта. Вам необходимо отфильтровать его по адресам страниц с текстами новостей. Известно, что шаблон страницы новостей имеет внутри url следующую конструкцию: /, затем 8 цифр, затем дефис. Выполните следующие действия:

Прочитайте содержимое файла с датафрейм
Отфильтруйте страницы с текстом новостей, используя метод str.contains и регулярное выражение в соответствии с заданным шаблоном

In [7]:
data_url = pd.read_csv("URLs.txt")
data_url[data_url.url.str.contains('[/]\d{8}[-]')]

,url
3,/politics/36188461-s-marta-zhizn-rossiyan-susc...
4,/world/36007585-tramp-pridumal-kak-reshit-ukra...
5,/science/36157853-nasa-sobiraet-ekstrennuyu-pr...
6,/video/36001498-poyavilis-pervye-podrobnosti-g...
7,/world/36007585-tramp-pridumal-kak-reshit-ukra...
...,...
88,/cis/35984145-kreml-prokommentiroval-soobschen...
89,/video/36071019-olimpiyskie-obekty-rio-prevrat...
90,/science/36151301-nazvano-posledstvie-zloupotr...
91,/incidents/36027330-vospitatelnitsu-zatravili-...


Задание 3

Используйте файл с оценками фильмов ml-latest-small/ratings.csv. Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным значением столбца timestamp для данного значения userId.

In [62]:
data_ratings = pd.read_csv('ratings.csv')
ratings_all = data_ratings.pivot_table(index = 'userId', columns = 'rating', values = 'timestamp', aggfunc = 'count', fill_value = 0, margins = True)
ratings_all_100 = ratings_all.loc[ratings_all['All'] > 100]
data_1 = ratings_all_100.rename_axis('userId').reset_index()
data_ratings.drop(columns=['movieId', 'rating'], axis=1, inplace=True)
data_max = data_ratings.loc[data_ratings.groupby('userId')['timestamp'].idxmax()]
data_max.rename(columns = {'timestamp': 'time_max'}, inplace=True)
data_min = data_ratings.loc[data_ratings.groupby('userId')['timestamp'].idxmin()]
data_min.rename(columns = {'timestamp': 'time_min'}, inplace=True)
data_result = pd.merge(data_min, data_max, how='left', on='userId') 
data_result['lifetime'] = data_result.time_max - data_result.time_min
print(data_result)


     userId    time_min    time_max  lifetime
0         1  1260759108  1260759205        97
1         2   835355395   835356246       851
2         3  1298861589  1298932787     71198
3         4   949778714   949982274    203560
4         5  1163373044  1163375145      2101
..      ...         ...         ...       ...
666     667   847271220   847272234      1014
667     668   993613196   993613478       282
668     669  1015829081  1015829766       685
669     670   938781328   940944033   2162705
670     671  1063500751  1074784735  11283984

[671 rows x 4 columns]


Задание 4

Дана статистика услуг перевозок клиентов компании по типам (см. файл “Python_13_join.ipynb” в разделе Материалы для лекции «Продвинутый pandas» ---- Ноутбуки к лекции «Продвинутый pandas»).
Необходимо сформировать две таблицы:
таблицу с тремя типами выручки для каждого client_id без указания адреса клиента
аналогичную таблицу по типам выручки с указанием адреса клиента

In [65]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)

In [66]:
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)

In [67]:

air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)

In [68]:
client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)

In [86]:
data1 = client_base.merge(air, how='left', on='client_id').merge(auto, how='left', on='client_id').merge(rzd, how='left', on='client_id')
data2 = data1.drop(columns=['address'], axis=1)

print(data1)
print(data2)

   client_id          address  air_revenue  auto_revenue  rzd_revenue
0        111  Комсомольская 4          NaN           NaN       1093.0
1        112   Энтузиастов 8а          NaN           NaN       2810.0
2        113  Левобережная 1а          NaN       57483.0      10283.0
3        114          Мира 14          NaN          83.0       5774.0
4        115        ЗЖБИиДК 1         81.0         912.0        981.0
5        116    Строителей 18          4.0        4834.0          NaN
6        117  Панфиловская 33         13.0          98.0          NaN
7        118     Мастеркова 4        173.0           NaN          NaN
   client_id  air_revenue  auto_revenue  rzd_revenue
0        111          NaN           NaN       1093.0
1        112          NaN           NaN       2810.0
2        113          NaN       57483.0      10283.0
3        114          NaN          83.0       5774.0
4        115         81.0         912.0        981.0
5        116          4.0        4834.0          NaN